In [1]:
path = '/home/hanna/miphclac/fractions_repo'

In [2]:
import glob
import os
import numpy as np
import xarray  as xr

In [3]:
files = glob.glob(os.path.join( path, '*2005*.nc' ))

In [4]:
np.sort(files)

array(['/home/hanna/miphclac/fractions_repo/2005_01.nc',
       '/home/hanna/miphclac/fractions_repo/2005_02.nc',
       '/home/hanna/miphclac/fractions_repo/2005_03.nc',
       '/home/hanna/miphclac/fractions_repo/2005_04.nc',
       '/home/hanna/miphclac/fractions_repo/2005_05.nc',
       '/home/hanna/miphclac/fractions_repo/2005_06.nc',
       '/home/hanna/miphclac/fractions_repo/2005_07.nc',
       '/home/hanna/miphclac/fractions_repo/2005_08.nc',
       '/home/hanna/miphclac/fractions_repo/2005_09.nc',
       '/home/hanna/miphclac/fractions_repo/2005_10.nc',
       '/home/hanna/miphclac/fractions_repo/2005_11.nc',
       '/home/hanna/miphclac/fractions_repo/2005_12.nc'], dtype='<U46')

In [8]:
jj = xr.open_mfdataset( files, concat_dim = 'time', combine = 'nested' )

In [13]:
jj.mean('time').tcc.values

array([[0.22207245, 0.22569004, 0.23204892, ..., 0.56235866, 0.5617735 ,
        0.56026675],
       [0.22567994, 0.22882253, 0.22764696, ..., 0.56035703, 0.55945608,
        0.55883134],
       [0.24306121, 0.23493628, 0.2371102 , ..., 0.55926627, 0.56151637,
        0.55934886],
       ...,
       [0.673265  , 0.66927497, 0.66967316, ..., 0.86466774, 0.8664386 ,
        0.86673411],
       [0.66676494, 0.66559801, 0.66311157, ..., 0.86918307, 0.86700679,
        0.87044205],
       [0.6526624 , 0.65238152, 0.64368441, ..., 0.86755982, 0.86911054,
        0.87389738]])

In [15]:
#import iris
import matplotlib.pyplot as plt

import cartopy.crs as ccrs


# load some sample iris data
#fname = iris.sample_data_path('rotated_pole.nc')
#temperature = iris.load_cube(fname)

# iris comes complete with a method to put bounds on a simple point
# coordinate. This is very useful...
temperature.coord('grid_latitude').guess_bounds()
temperature.coord('grid_longitude').guess_bounds()

# turn the iris Cube data structure into numpy arrays
gridlons = temperature.coord('grid_longitude').contiguous_bounds()
gridlats = temperature.coord('grid_latitude').contiguous_bounds()
temperature = temperature.data

# set up a map
ax = plt.axes(projection=ccrs.PlateCarree())

# define the coordinate system that the grid lons and grid lats are on
rotated_pole = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)
plt.pcolormesh(gridlons, gridlats, temperature, transform=rotated_pole)

ax.coastlines()

plt.show()

ModuleNotFoundError: No module named 'cartopy'

In [36]:
temp

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 161, time: 720)
Coordinates:
  * time       (time) datetime64[ns] 2005-11-01 ... 2005-11-30T23:00:00
  * latitude   (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * longitude  (longitude) float64 -15.0 -14.75 -14.5 -14.25 ... 24.5 24.75 25.0
Data variables:
    tcc        (time, latitude, longitude) float64 ...
    nr_nans    (time, latitude, longitude) float64 ...

In [41]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 161, time: 743)
Coordinates:
  * time       (time) datetime64[ns] 2005-12-01 ... 2005-12-31T23:00:00
  * latitude   (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * longitude  (longitude) float64 -15.0 -14.75 -14.5 -14.25 ... 24.5 24.75 25.0
Data variables:
    tcc        (time, latitude, longitude) float64 ...
    nr_nans    (time, latitude, longitude) float64 ...

In [45]:
xr.merge([ds, temp])

RuntimeError: NetCDF: HDF error

In [38]:
for i, fil in enumerate(files):
    if i ==0:
        ds = xr.open_dataset(fil)
    else:
        temp =  xr.open_dataset(fil)
        #ds = temp.merge(ds, 'time')

TypeError: The `inplace` argument has been removed from xarray. You can achieve an identical effect with python's standard assignment.

In [30]:
def open_multiple_files(files, concat_dim = 'time'):
    """ Returns a dataset concatinated from multiple files. """
    for i, fil in enumerate(files):
        if i ==0:
            ds = xr.open_dataset(fil)
        else:
            ds = ds.merge( xr.open_dataset(fil) )
    return ds
        

In [31]:
ds = open_multiple_files(files)

RuntimeError: NetCDF: HDF error

In [20]:
data = xr.open_mfdataset(files, concat_dim = 'time')

/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  """Entry point for launching an IPython kernel.
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/xarray/backends/api.py:931: FutureWarning: Also `open_mfdataset` will no longer accept a `concat_dim` argument.
To get equivalent behaviour from now on please use the new
`combine_nested` function instead (or the `combine='nested'` option to
`open_mfdataset`).The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating 

In [29]:
data.tcc.

<xarray.DataArray 'tcc' (time: 8630, latitude: 81, longitude: 161)>
dask.array<concatenate, shape=(8630, 81, 161), dtype=float64, chunksize=(744, 81, 161), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float64 -15.0 -14.75 -14.5 -14.25 ... 24.5 24.75 25.0
  * latitude   (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * time       (time) datetime64[ns] 2005-12-01 ... 2005-04-30T23:00:00

In [ ]:
hrs_in_year = 